In [86]:
import pandas as pd
import mariadb
import numpy as np
from tqdm import tqdm

In [87]:
# mariaDB 접속값
conn_params = {'host':'localhost',
               'port':3306,
               'user':'root',
               'password':'root',
               'database':"musicdb",
               'autocommit':True}
conn = mariadb.connect(**conn_params)
cursor = conn.cursor()

# DB 에서 musictb 데이터 조회하는  sql 실행
cursor.execute('select * from musictb;')

# 조회결과 가져오기
queryDatas=cursor.fetchall()

df=pd.DataFrame(queryDatas)

cursor.close()
conn.close()

In [88]:
# 가져온 df 확인
df.columns=['가수','제목','앨범사진','리뷰1','리뷰2']

In [89]:
df['총리뷰']=df['리뷰1'].str.replace("|"," ")+" "+df['리뷰2'].str.replace("|"," ")

In [90]:
# 정규 표현식을 통한 한글 외 문자 제거
import re
df['총리뷰'] = df['총리뷰'].apply(lambda x: re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', x))

In [91]:
from konlpy.tag import Okt

# 토큰화
okt=Okt()

# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다','재생','다운로드','곡명','앨범명','아티스트명','ㅠ','ㅋ','왈',
             'ㄱㄱ', 'ㄱㄱㄱ','ㄱㄱㄹ',	'ㄱㄱㅂ', 'ㄱㄲ','ㄱㄴ', 'ㄱㄴㄴ','ㄱㄴㄷㄹ','ㄱㄴㄷㄹㅁ','ㄱㄴㄷㄹㅁㅂ']

# 명사와 형용사만 선택
tokenized_data = []
for s in tqdm(df['총리뷰']):
    sentence = okt.pos(s, norm=True, # norm 정규화
                        stem=True)  # 형태소와 품사 정보 추출

    # 조사 삭제
    selected_words =[word for word, pos in sentence if pos != 'Josa' and word not in stopwords]
    tokenized_data.append(" ".join(selected_words))

  0%|          | 0/578 [00:00<?, ?it/s]

100%|██████████| 578/578 [03:04<00:00,  3.14it/s]


In [93]:
# 명사,형용사만 있는 새로운 컬럼 추가
df['token_review']=tokenized_data

In [94]:
# csv 파일로 저장
df.to_csv('./not_josa.csv',encoding='utf-8')